### Recommender Module

In [ ]:
import re
import pandas as pd

# Load datasets
foundation_data = pd.read_excel("Make-Up Recommendation.xlsx", sheet_name="Foundation")
lipstick_data   = pd.read_excel("Make-Up Recommendation.xlsx", sheet_name="Lipstick")

COLS = {
    "season": "Suitable for which weather",
    "skin":   "Use for which Skin Type",
    "brand":  "Brand Name",
    "name":   "Product Name",
    "price":  "Price",
    "rating": "Ratings",
}

def _filter_and_serialize(df, season, skin_type, top_n=5):
    # Normalize the text columns we'll filter on
    season_col = COLS["season"]
    skin_col   = COLS["skin"]
    for c in [season_col, skin_col]:
        df[c] = df[c].astype(str).str.strip()

    # Word-boundary contains (handles "Summer, Monsoon" / "Oily, Combination")
    season_pat = rf"\b{re.escape(season.strip())}\b"
    skin_pat   = rf"\b{re.escape(skin_type.strip())}\b"

    season_mask = df[season_col].str.contains(season_pat, case=False, na=False)
    skin_mask   = df[skin_col].str.contains(skin_pat, case=False, na=False)
    filtered = df[season_mask & skin_mask].copy()

    # Sort: Ratings desc, Price asc
    if COLS["rating"] in filtered.columns:
        filtered[COLS["rating"]] = pd.to_numeric(filtered[COLS["rating"]], errors="coerce")
    if COLS["price"] in filtered.columns:
        filtered[COLS["price"]] = pd.to_numeric(filtered[COLS["price"]], errors="coerce")

    filtered = filtered.sort_values(by=[COLS["rating"], COLS["price"]], ascending=[False, True])

    if filtered.empty:
        return "[]"

    keep = [c for c in [COLS["brand"], COLS["name"], COLS["price"], COLS["rating"]] if c in filtered.columns]
    # Take top N after sorting
    top = filtered[keep].head(int(top_n))
    return top.to_json(orient="records")

def recommend_lipstick(season, skin_type, top_n=5):
    return _filter_and_serialize(lipstick_data, season, skin_type, top_n=top_n)

def recommend_foundation(season, skin_type, top_n=5):
    return _filter_and_serialize(foundation_data, season, skin_type, top_n=top_n)

